In [15]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import tensorboard
import optuna

# Install tqdm if not already installed
from tqdm.autonotebook import tqdm as notebook_tqdm

In [16]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward, contact_cost_weight, healthy_z_range, contact_force_range):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    contact_cost_weight = contact_cost_weight,
                    healthy_z_range=healthy_z_range,
                    contact_force_range=contact_force_range)
                   # render_mode='none')

In [17]:
    # reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.05, 0.2)           # Default circa 0.1; esploriamo da 0.05 a 0.2
    # forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.5, 1.5)     # Default tipico è 1; esploriamo da 0.5 a 1.5
    # ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.1, 1.0)               # Default tipico 0.5; esploriamo da 0.1 a 1.0
    # healthy_reward = trial.suggest_float('healthy_reward', 0.5, 1.5)                   # Default tipico 1; esploriamo da 0.5 a 1.5
    
    # # Parametri aggiuntivi per Ant-v5
    # contact_cost_weight = trial.suggest_float('contact_cost_weight', 1e-4, 1e-3)  # Es. range intorno a 5e-4 come default
    # healthy_z_lower = trial.suggest_float('healthy_z_lower', 0.1, 0.3)             # Per definire l'intervallo di altezze "sane"
    # healthy_z_upper = trial.suggest_float('healthy_z_upper', 0.8, 1.2)
    # contact_force_min = trial.suggest_float('contact_force_min', -1.0, -0.5)         # Modificabile se usi forze di contatto
    # contact_force_max = trial.suggest_float('contact_force_max', 0.5, 1.0)


    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    # n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    # batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])  
    # # Per ambienti complessi come Ant, molti esperimenti usano gamma intorno a 0.99-0.995
    # gamma = trial.suggest_float('gamma', 0.99, 0.999)
    # gae_lambda = trial.suggest_float('gae_lambda', 0.8, 1.0)
    # clip_range = trial.suggest_float('clip_range', 0.1, 0.3) 
    # ent_coef = trial.suggest_float('ent_coef', 0.0, 0.1)

In [18]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.05, 0.2)           # Default circa 0.1; esploriamo da 0.05 a 0.2
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.5, 1.5)     # Default tipico è 1; esploriamo da 0.5 a 1.5
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.1, 1.0)               # Default tipico 0.5; esploriamo da 0.1 a 1.0
    healthy_reward = trial.suggest_float('healthy_reward', 0.5, 1.5)                   # Default tipico 1; esploriamo da 0.5 a 1.5
    
    # Parametri aggiuntivi per Ant-v5
    contact_cost_weight = trial.suggest_float('contact_cost_weight', 1e-4, 1e-3)  # Es. range intorno a 5e-4 come default
    healthy_z_lower = trial.suggest_float('healthy_z_lower', 0.1, 0.3)             # Per definire l'intervallo di altezze "sane"
    healthy_z_upper = trial.suggest_float('healthy_z_upper', 0.8, 1.2)
    contact_force_min = trial.suggest_float('contact_force_min', -1.0, -0.5)         # Modificabile se usi forze di contatto
    contact_force_max = trial.suggest_float('contact_force_max', 0.5, 1.0)

    # Crea l'ambiente passando tutti i parametri
    env = make_env(
        reset_noise_scale,
        forward_reward_weight,
        ctrl_cost_weight,
        healthy_reward,
        contact_cost_weight=contact_cost_weight,
        healthy_z_range=(healthy_z_lower, healthy_z_upper),
        contact_force_range=(contact_force_min, contact_force_max)
    )
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Iperparametri per il modello PPO
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    batch_size = trial.suggest_categorical('batch_size', [512, 1024, 2048, 4096])  
    # Per ambienti complessi come Ant, molti esperimenti usano gamma intorno a 0.99-0.995
    gamma = trial.suggest_float('gamma', 0.965, 0.98)
    gae_lambda = trial.suggest_float('gae_lambda', 0.9, 1.0)
    clip_range = trial.suggest_float('clip_range', 0.3, 0.5) 
    ent_coef = trial.suggest_float('ent_coef', 0.0, 0.1)
    
    # Nuovo iperparametro per la penalizzazione della varianza
    std_penalty_weight = trial.suggest_float('std_penalty_weight', 0.0, 0.5)

    # Crea ed allena il modello PPO
    model = PPO("MlpPolicy", env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range,
                ent_coef=ent_coef,
                seed=42,
                verbose=0)
    model.learn(total_timesteps=200000)

    # Valuta il modello su 200 episodi (200 è ottimale)
    episodes = 150

    episode_rewards = []
    for episode in range(episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)

    # Calcola reward media e varianza
    mean_reward = np.mean(episode_rewards)
    reward_std = np.std(episode_rewards)

    # Definisce l'obiettivo: massimizzare la reward media penalizzando la varianza
    score = mean_reward - std_penalty_weight * reward_std

    print(f'Mean is: {mean_reward}, Std is: {reward_std}\n')

    return score

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)

[I 2025-02-12 17:51:23,514] A new study created in memory with name: no-name-1299e639-7b0b-4f75-861b-91da70fccdd3
/var/folders/5w/qb_kxxjs5lscg9_8tpttrzs40000gn/T/ipykernel_2129/1646536018.py:31: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[I 2025-02-12 17:52:31,664] Trial 0 finished with value: -2.2595818042755127 and parameters: {'reset_noise_scale': 0.10550150920629535, 'forward_reward_weight': 1.1310290950229411, 'ctrl_cost_weight': 0.656175542456229, 'healthy_reward': 1.402054053468586, 'contact_cost_weight': 0.00011460398761631547, 'healthy_z_lower': 0.27796109976617994, 'healthy_z_upper': 1.102818142370627, 'contact_force_min': -0.9224021861038963, 'contact_force_max': 0.8865754974915587, 'learning_rate': 5.332593661181692e-05, 'n_steps': 2048, 'batc

Mean is: -2.0479979515075684, Std is: 1.8384965658187866



[I 2025-02-12 17:53:52,547] Trial 1 finished with value: 6.8863396644592285 and parameters: {'reset_noise_scale': 0.061516859181143226, 'forward_reward_weight': 0.9468327954193553, 'ctrl_cost_weight': 0.2170985768679085, 'healthy_reward': 1.2922614989484824, 'contact_cost_weight': 0.0005849111914860173, 'healthy_z_lower': 0.2044310032992321, 'healthy_z_upper': 1.137775407579622, 'contact_force_min': -0.845824754293711, 'contact_force_max': 0.7440804407728538, 'learning_rate': 9.278620006867075e-05, 'n_steps': 8192, 'batch_size': 4096, 'gamma': 0.9718478602122956, 'gae_lambda': 0.9010149714316421, 'clip_range': 0.3036494041215037, 'ent_coef': 0.06470603172295075, 'std_penalty_weight': 0.011711804278705829}. Best is trial 1 with value: 6.8863396644592285.


Mean is: 7.000239372253418, Std is: 9.725189208984375



[I 2025-02-12 17:55:03,039] Trial 2 finished with value: -5.570282936096191 and parameters: {'reset_noise_scale': 0.10351098208385554, 'forward_reward_weight': 1.0461443882426187, 'ctrl_cost_weight': 0.3290557766230068, 'healthy_reward': 0.6409966912089246, 'contact_cost_weight': 0.0007437459013269457, 'healthy_z_lower': 0.16176842222262425, 'healthy_z_upper': 1.1319821491367066, 'contact_force_min': -0.5680651007429471, 'contact_force_max': 0.7916323856222833, 'learning_rate': 0.000411221496590389, 'n_steps': 4096, 'batch_size': 1024, 'gamma': 0.9707153321548935, 'gae_lambda': 0.9163930904551032, 'clip_range': 0.47904058916669406, 'ent_coef': 0.07546370422744655, 'std_penalty_weight': 0.4224321898813669}. Best is trial 1 with value: 6.8863396644592285.


...

Mean is: 28.410146713256836, Std is: 18.702049255371094



[I 2025-02-12 23:05:36,542] Trial 206 finished with value: 25.78643035888672 and parameters: {'reset_noise_scale': 0.11363405960496634, 'forward_reward_weight': 0.6532737274304571, 'ctrl_cost_weight': 0.1313728476643531, 'healthy_reward': 1.3726897043561155, 'contact_cost_weight': 0.0005511399748709426, 'healthy_z_lower': 0.19038253926035162, 'healthy_z_upper': 1.1901249434359422, 'contact_force_min': -0.6647768348139104, 'contact_force_max': 0.7548419487433393, 'learning_rate': 0.0002736453787602979, 'n_steps': 2048, 'batch_size': 512, 'gamma': 0.9733920287037915, 'gae_lambda': 0.9266441263277047, 'clip_range': 0.4913114123346966, 'ent_coef': 0.006002670544466264, 'std_penalty_weight': 0.12324476233908144}. Best is trial 148 with value: 30.901912689208984.


Mean is: 28.044198989868164, Std is: 18.319379806518555



[W 2025-02-12 23:05:55,460] Trial 207 failed with parameters: {'reset_noise_scale': 0.09882745989657987, 'forward_reward_weight': 0.680277644968093, 'ctrl_cost_weight': 0.1490959442918867, 'healthy_reward': 1.4093760313974195, 'contact_cost_weight': 0.000488185341807725, 'healthy_z_lower': 0.20905480903441637, 'healthy_z_upper': 1.178642228071086, 'contact_force_min': -0.6453245438172286, 'contact_force_max': 0.751388572225389, 'learning_rate': 0.00030861786742803973, 'n_steps': 2048, 'batch_size': 512, 'gamma': 0.9732543352306958, 'gae_lambda': 0.932294006945299, 'clip_range': 0.4878404506064939, 'ent_coef': 0.011189992706647924, 'std_penalty_weight': 0.16997855031576461} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/Reinforcement_Learning_MuJoCu/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)


KeyboardInterrupt: 